In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import bioinfokit.analys
from scipy import stats 
from sklearn.metrics import mean_absolute_error as score_MAE
from sklearn.metrics import mean_squared_error as score_MSE 
from sklearn.metrics import median_absolute_error as score_MED
from sklearn.metrics import r2_score as score_r2
import matplotlib.gridspec as gridspec

In [ ]:
###  Macaca fascicularis batch-corrected CPM
file = './source_data/batch_correction/batch_cor_GeneCPM_mfa.txt'
GeneCPM = pd.read_csv(file, sep = '\t', header = 0)
GeneCPM = GeneCPM.set_index('gene_chr')

###  Macaca mulatta CPM
#file = './source_data/count_result/GeneCPM_mml.txt'
file = './source_data/count_result/GeneCPM_mml.txt'
GeneCPM_mml = pd.read_csv(file, sep = '\t', header = 0)
GeneCPM_mml = GeneCPM_mml.set_index('gene_chr')

#########################  HUMAN_Mendeley_2020_data
#### Genecount_CPM
Men_gene_read = pd.read_csv('./source_data/Mendeley_2020/raw_filtered.txt', sep = '\t', header = 0)
Men_gene_read = Men_gene_read.rename(columns={'Unnamed: 0':'Name'})

Men_sample = pd.read_csv('./source_data/Mendeley_2020/meta_filtered.txt', sep = '\t', header = 0)

#########################  TRAP_GENE

TRAP = pd.read_excel('./source_data/AGE_prediction/TRAP_R/41467_2015_BFncomms9570_MOESM436_ESM.xlsx',header=0,skiprows=2)
#TRAP = TRAP.reset_index(drop=True)

TRAP = TRAP[['NEW-Gene-ID','P','META: P']]
TRAP = TRAP.rename(columns={'NEW-Gene-ID':'gene'})
TRAP.to_csv('./source_data/AGE_prediction/TRAP_R/TRAP.txt',sep='\t',index=True,header=True)

TRAP_cyno = pd.read_csv('./source_data/AGE_prediction/TRAP_R/TRAP_cyno.txt', sep = '\t', header = 0)
TRAP_rhesus = pd.read_csv('./source_data/AGE_prediction/TRAP_R/TRAP_rhesus.txt', sep = '\t', header = 0)

TRAP['disc_fdr'] = TRAP['P']*len(TRAP) / (TRAP.index+1)
TRAP['meta_fdr'] = TRAP['META: P']*len(TRAP) / (TRAP.index+1)
TRAP_GENE_DISC = TRAP[TRAP['disc_fdr'] <= 0.1**3]
TRAP_GENE_META = TRAP[TRAP['meta_fdr'] <= 0.1**3]

TRAP_cyno['disc_fdr'] = TRAP_cyno['P']*len(TRAP_cyno) / (TRAP_cyno.index+1)
TRAP_cyno['meta_fdr'] = TRAP_cyno['META..P']*len(TRAP_cyno) / (TRAP_cyno.index+1)
TRAP_cyno_GENE_DISC = TRAP_cyno[TRAP_cyno['disc_fdr'] <= 0.1**3]
TRAP_cyno_GENE_META = TRAP_cyno[TRAP_cyno['meta_fdr'] <= 0.1**3]

TRAP_rhesus['disc_fdr'] = TRAP_rhesus['P']*len(TRAP_rhesus) / (TRAP_rhesus.index+1)
TRAP_rhesus['meta_fdr'] = TRAP_rhesus['META..P']*len(TRAP_rhesus) / (TRAP_rhesus.index+1)
TRAP_rhesus_GENE_DISC = TRAP_rhesus[TRAP_rhesus['disc_fdr'] <= 0.1**3]
TRAP_rhesus_GENE_META = TRAP_rhesus[TRAP_rhesus['meta_fdr'] <= 0.1**3]

#########################  PREDICTOR
predictor = pd.read_excel('./source_data/AGE_prediction/TRAP_R/41467_2015_BFncomms9570_MOESM440_ESM.xlsx', sheet_name='Data.5.B',skiprows=3)
predictor = predictor[['GeneID','PREDICTOR-for-NEW-COHORTS']]

In [ ]:
#########################  AGE

### Macaca fascicularis

file = './source_data/reference/mfa_samplelist.txt'
mfa_Con = pd.read_csv(file, sep = '\t', header = None, names=['SAMPLEID'])
mfa_Con['AGE'] = [3,3,7,7,12,12,17,17,4,4,8,8,13,13,18,18,5,5,9,9,14,14,19,19]
mfa_Con = mfa_Con.sort_values(by=['AGE','SAMPLEID'], ascending=[True,True])
#mfa_Con.to_csv('/home/chm/analyzing_file_byR/RNA/SQuIRE_count_ELTC/AGE_predition/TRAP_R/SAMPLEID_AGE.txt',sep='\t',index=False,header=True)

GeneCPM = GeneCPM[mfa_Con['SAMPLEID'].tolist()]

### Macaca mulatta

file = './source_data/reference/mml_condition.txt'
mml_Con = pd.read_csv(file, sep = ' ', header = None, names=['SAMPLEID','AGE','ID','SEX'])

#mml_Con = mml_Con[mml_Con['SEX']=='male']
mml_Con = mml_Con[['SAMPLEID','AGE']]
mml_Con = mml_Con.sort_values(by=['AGE','SAMPLEID'], ascending=[True,True])

GeneCPM_mml = GeneCPM_mml[mml_Con['SAMPLEID'].tolist()]

In [ ]:
#########################  AGE

### Homo sapience

GeneRaw_hsa = Men_gene_read.rename(columns={'Unnamed: 0':'Name'})
GeneRaw_hsa = GeneRaw_hsa.set_index('Name')

### 

#SEL_healthy = Men_sample[(Men_sample['Gender'] == 'Male') &(Men_sample['Tissue'].str.contains('Blood;PBMC')) & (Men_sample['Condition'] == 'Healthy')]
#SEL_healthy = Men_sample[(Men_sample['Tissue'].str.contains('Blood;PBMC')) & (Men_sample['Condition'] == 'Healthy')]
#SEL_healthy = Men_sample[(Men_sample['Tissue'].str.contains('Blood;PBMC')) & (Men_sample['Condition'] == 'Healthy')]

SEL_healthy = Men_sample[(Men_sample['Tissue'].str.contains('Blood')) & (Men_sample['Condition'] == 'Healthy')]
SEL_unhealthy = Men_sample[(Men_sample['Tissue'].str.contains('Blood')) & (~(Men_sample['Condition'] == 'Healthy'))]

print(len(SEL_healthy))
print(len(SEL_unhealthy))

#SEL_healthy = Men_sample[(Men_sample['Condition'] == 'Healthy')]
#SEL_unhealthy = Men_sample[~(Men_sample['Condition'] == 'Healthy')]

h_hsa_Con = SEL_healthy[['SRR.ID','Age']]
h_hsa_Con = h_hsa_Con.rename(columns={'SRR.ID':'SAMPLEID','Age':'AGE'})
GeneRaw_hhsa = GeneRaw_hsa[h_hsa_Con['SAMPLEID'].tolist()]

uh_hsa_Con = SEL_unhealthy[['SRR.ID','Age']]
uh_hsa_Con = uh_hsa_Con.rename(columns={'SRR.ID':'SAMPLEID','Age':'AGE'})
GeneRaw_huhsa = GeneRaw_hsa[uh_hsa_Con['SAMPLEID'].tolist()]

### CPM_normalization

nm = bioinfokit.analys.norm()
nm.cpm(df=GeneRaw_hhsa)
GeneRaw_hhsa = nm.cpm_norm

nm = bioinfokit.analys.norm()
nm.cpm(df=GeneRaw_huhsa)
GeneRaw_huhsa = nm.cpm_norm

In [ ]:
hubgene = pd.read_csv('./source_data/hubgene/hubgene_20.txt', sep = '\t', header = 0)
hubgene_all = pd.read_csv('./source_data/hubgene/hubgene_605.txt', sep = '\t', header = 0)
hubgene_cancer = pd.read_csv('./source_data/hubgene/hubgene_cancer.txt', sep = '\t', header = 0)

hubgene_rhesus = pd.read_csv('./source_data/hubgene/biomaRt/hubgene_20_rhesus.txt', sep = '\t', header = 0)
hubgene_all_rhesus = pd.read_csv('./source_data/hubgene/biomaRt/hubgene_605_rhesus.txt', sep = '\t', header = 0)
hubgene_cancer_rhesus =  pd.read_csv('./source_data/hubgene/biomaRt/hubgene_cancer_rhesus.txt', sep = '\t', header = 0)

hubgene_human = pd.read_csv('./source_data/hubgene/biomaRt/hubgene_20_human.txt', sep = '\t', header = 0)
hubgene_all_human = pd.read_csv('./source_data/hubgene/biomaRt/hubgene_605_human.txt', sep = '\t', header = 0)
hubgene_cancer_human =  pd.read_csv('./source_data/hubgene/biomaRt/hubgene_cancer_human.txt', sep = '\t', header = 0)

In [ ]:
def formula(dt_T):

    DF = pd.DataFrame()

    for i in predictor.index:
        gene = predictor.loc[i]['GeneID']
        value = predictor.loc[i]['PREDICTOR-for-NEW-COHORTS']
    
        if gene in dt_T.columns:
            DF[gene] = dt_T[gene] * value
        else:
            continue
    return DF

def parsing_formula(dt,dt_filterlist):
    
    dt['GENEID'] = dt.index.str.split('_').str[0]
    dt = dt.set_index('GENEID')
    dt = dt.reset_index(drop=False)
    
    dt = dt.groupby(['GENEID'])[dt.columns.tolist()].sum()
    dt = dt.reset_index(drop=False)

    dt = dt[dt['GENEID'].isin(dt_filterlist)]
    dt = dt.set_index('GENEID')
    dt_T = dt.T
    
    dt_T[dt_T < 2] = 0
    
    dt_T_f = formula(dt_T) 
    
    return dt_T_f

In [ ]:
D_mfa = parsing_formula(GeneCPM,TRAP_cyno_GENE_DISC['gene'].unique())
M_mfa = parsing_formula(GeneCPM,TRAP_cyno_GENE_META['gene'].unique())

D_mml = parsing_formula(GeneCPM_mml,TRAP_rhesus_GENE_DISC['gene'].unique())
M_mml = parsing_formula(GeneCPM_mml,TRAP_rhesus_GENE_META['gene'].unique())

ND_hsa = parsing_formula(GeneRaw_hhsa,TRAP_GENE_DISC['gene'].unique())
NM_hsa = parsing_formula(GeneRaw_hhsa,TRAP_GENE_META['gene'].unique())

AND_hsa = parsing_formula(GeneRaw_huhsa,TRAP_GENE_DISC['gene'].unique())
ANM_hsa = parsing_formula(GeneRaw_huhsa,TRAP_GENE_META['gene'].unique())

In [ ]:
def mfa_mml_hub(df,df_rhesus):

    its = list(set(predictor['GeneID'].unique()) & set(df['gene'].tolist()))
    mfa_hub = parsing_formula(GeneCPM,its)

    its = list(set(predictor['GeneID'].unique()) & set(df['gene'].tolist()))
    mml_hub = parsing_formula(GeneCPM_mml,its)

    
    return mfa_hub, mml_hub

def hsa_hub(df):

    its = list(set(predictor['GeneID'].unique()) & set(df['gene'].tolist()))
    nha_hub = parsing_formula(GeneRaw_hhsa,its)

    return nha_hub

def uhsa_hub(df):

    its = list(set(predictor['GeneID'].unique()) & set(df['gene'].tolist()))
    uha_hub = parsing_formula(GeneRaw_huhsa,its)

    return uha_hub

mfa_hubgene_cancer, mml_hubgene_cancer = mfa_mml_hub(hubgene_cancer,hubgene_cancer_rhesus)
mfa_hubgene, mml_hubgene = mfa_mml_hub(hubgene,hubgene_rhesus)
mfa_hubgene_all, mml_hubgene_all = mfa_mml_hub(hubgene_all,hubgene_all_rhesus)

nha_hubgene_cancer = hsa_hub(hubgene_cancer_human)
nha_hubgene = hsa_hub(hubgene_human)
nha_hubgene_all = hsa_hub(hubgene_all_human)

uha_hubgene_cancer = uhsa_hub(hubgene_cancer_human)
uha_hubgene = uhsa_hub(hubgene_human)
uha_hubgene_all = uhsa_hub(hubgene_all_human)

In [ ]:
def prediction(dt,chron_age):

    dt_sum = dt.sum(axis=1)
    
    mean_pred = dt_sum.mean()
    std_pred = dt_sum.std()
 
    mean_chron = chron_age['AGE'].mean()
    std_chron = chron_age['AGE'].std()

    age = []
    for i in dt_sum.index:
        predictor = dt_sum.loc[i]
        pred_age = mean_chron + (predictor-mean_pred) * (std_chron / std_pred)
        age.append(pred_age)
        
    return age



def merge(a,b,c,d,e,con):

    con['hub_age'] = prediction(a,con)
    con['hub_age_cancer'] = prediction(b,con)
    con['hub_age_all'] = prediction(c,con)
    con['DISC_age'] = prediction(d,con)
    con['META_age'] = prediction(e,con)  
    
    return con


###
MF = merge(mfa_hubgene,mfa_hubgene_cancer,mfa_hubgene_all,D_mfa,M_mfa,mfa_Con)
ML = merge(mml_hubgene,mml_hubgene_cancer,mml_hubgene_all,D_mml,M_mml,mml_Con) 

NH = merge(nha_hubgene,nha_hubgene_cancer,nha_hubgene_all,ND_hsa,NM_hsa,h_hsa_Con) 
AH = merge(uha_hubgene,uha_hubgene_cancer,uha_hubgene_all,AND_hsa,ANM_hsa,uh_hsa_Con) 

In [ ]:
def age_graph(dt,min_age,max_age,Y_max,S):
    
    true_age = dt['AGE']
    pred_age_1 = dt['DISC_age']
    pred_age_2 = dt['META_age']
    pred_age_3 = dt['hub_age_cancer']
    pred_age_4 = dt['hub_age']
    pred_age_5 = dt['hub_age_all']
    
    r1, p1 = stats.pearsonr(true_age,pred_age_1)
    r2, p2 = stats.pearsonr(true_age,pred_age_2)
    r3, p3 = stats.pearsonr(true_age,pred_age_3)
    r4, p4 = stats.pearsonr(true_age,pred_age_4)
    r5, p5 = stats.pearsonr(true_age,pred_age_5)

    err_1 = "MAE:{:3.1f}  MED:{:3.1f} R:{:3.2f}".format( score_MAE(true_age,pred_age_1), score_MED(true_age,pred_age_1), r1)
    err_2 = "MAE:{:3.1f}  MED:{:3.1f} R:{:3.2f}".format( score_MAE(true_age,pred_age_2), score_MED(true_age,pred_age_2), r2)
    err_3 = "MAE:{:3.1f}  MED:{:3.1f} R:{:3.2f}".format( score_MAE(true_age,pred_age_3), score_MED(true_age,pred_age_3), r3)
    err_4 = "MAE:{:3.1f}  MED:{:3.1f} R:{:3.2f}".format( score_MAE(true_age,pred_age_4), score_MED(true_age,pred_age_4), r4)    
    err_5 = "MAE:{:3.1f}  MED:{:3.1f} R:{:3.2f}".format( score_MAE(true_age,pred_age_5), score_MED(true_age,pred_age_5), r5)   

    
    sns.set_style('white')
    sns.set_palette( sns.xkcd_palette(['nice blue','faded red']))
    
    fig = plt.figure(figsize=(15,3.3))
    #fig = plt.figure(figsize=(13,3.5))
    
    ax1 = fig.add_subplot(1,5,1)
    ax1.plot([3,max_age],[3,max_age],'k:')

    sns.regplot(data=dt, x='AGE', y='DISC_age', ax=ax1, color='orange')
    
    if S == 'Macaca fascicularis' or S == 'Macaca mulatta':
        ax1.text(0.95, 0.03, err_1, horizontalalignment='right',verticalalignment='bottom',transform=ax1.transAxes)
    else:
        ax1.text(1,0, err_1, horizontalalignment='right',verticalalignment='bottom',transform=ax1.transAxes)

    if S == 'Macaca fascicularis':
        ax1.text(12,25,'DISC', ha='center', va='center', size=20,color = 'orange')
    if S == 'Normal Human':
        ax1.text(35,60,'DISC', ha='center', va='center', size=20,color = 'orange')
        
    if S == 'Macaca fascicularis' or S == 'Macaca mulatta':
        ax1.text(-7, 12, S , ha='center', va='center', size=20,  rotation=90)
    else:
        ax1.text(-30, 50, S , ha='center', va='center', size=20,  rotation=90)

    plt.axis('equal')
    ax1.set_xlabel('True age (years)')
    ax1.set_ylabel('Predicted age (years)')
    sns.despine()
    if S == 'Macaca fascicularis' or S == 'Macaca mulatta':
        plt.ylim(0,25)       
    else:
        plt.ylim(20,Y_max)       
    plt.xlim(min_age,max_age)
    plt.tight_layout()
    

    ax2 = fig.add_subplot(1,5,2)
    ax2.plot([3,max_age],[3,max_age],'k:')

    sns.regplot(data=dt, x='AGE', y='META_age', ax=ax2, color='deepskyblue')
    
    
    if S == 'Macaca fascicularis' or S == 'Macaca mulatta':
        ax2.text(0.95, 0.03, err_2, horizontalalignment='right',verticalalignment='bottom',transform=ax2.transAxes)
    else:
        ax2.text(1, 0, err_2, horizontalalignment='right',verticalalignment='bottom',transform=ax2.transAxes)

    if S == 'Macaca fascicularis':
        ax2.text(11.9,25,'META', ha='center', va='center', size=20,color = 'deepskyblue')
    if S == 'Normal Human':
        ax2.text(35,56,'META', ha='center', va='center', size=20,color = 'deepskyblue')

    plt.axis('equal')
    ax2.set_xlabel('True age (years)')
    #ax2.set_ylabel('Predicted age (years)')
    plt.ylabel('')
    if S == 'Macaca fascicularis' or S == 'Macaca mulatta':
        plt.ylim(0,25)       
    else:
        plt.ylim(20,Y_max)       
    plt.xlim(min_age,max_age)
    sns.despine()
    plt.tight_layout()
    
    
    ax3 = fig.add_subplot(1,5,3)
    ax3.plot([3,max_age],[3,max_age],'k:')

    sns.regplot(data=dt, x='AGE', y='hub_age_cancer', ax=ax3, color='darksalmon')
    
    if S == 'Macaca fascicularis' or S == 'Macaca mulatta':
        ax3.text(0.95, 0.03, err_3, horizontalalignment='right',verticalalignment='bottom',transform=ax3.transAxes)
    else:
        ax3.text(1,0, err_3, horizontalalignment='right',verticalalignment='bottom',transform=ax3.transAxes)
    
    if S == 'Macaca fascicularis':
        ax3.text(14,25,'HUB_cancer', ha='center', va='center', size=20,color = 'darksalmon')
    if S == 'Normal Human':
        ax3.text(36,56,'HUB_cancer', ha='center', va='center', size=20,color = 'darksalmon')
   
    plt.axis('equal')
    ax3.set_xlabel('True age (years)')
    #ax3.set_ylabel('Predicted age (years)')
    plt.ylabel('')
    if S == 'Macaca fascicularis' or S == 'Macaca mulatta':
        plt.ylim(0,25)       
    else:
        plt.ylim(20,Y_max)       
    plt.xlim(min_age,max_age)
    sns.despine()
    plt.tight_layout()
    
    
    ax4 = fig.add_subplot(1,5,4)
    ax4.plot([3,max_age],[3,max_age],'k:')

    sns.regplot(data=dt, x='AGE', y='hub_age', ax=ax4, color='darksalmon')
    
    
    if S == 'Macaca fascicularis' or S == 'Macaca mulatta':
        ax4.text(0.95, 0.03, err_4, horizontalalignment='right',verticalalignment='bottom',transform=ax4.transAxes)
    else:
        ax4.text(1,0, err_4, horizontalalignment='right',verticalalignment='bottom',transform=ax4.transAxes)
  
    if S == 'Macaca fascicularis':
        ax4.text(12,25,'HUB_20', ha='center', va='center', size=20,color = 'darksalmon')
    if S == 'Normal Human':
        ax4.text(35,56,'HUB_20', ha='center', va='center', size=20,color = 'darksalmon')
   
    plt.axis('equal')
    ax4.set_xlabel('True age (years)')
    #ax4.set_ylabel('Predicted age (years)')
    plt.ylabel('')
    if S == 'Macaca fascicularis' or S == 'Macaca mulatta':
        plt.ylim(0,25)       
    else:
        plt.ylim(20,Y_max)       
    plt.xlim(min_age,max_age)
    sns.despine()
    plt.tight_layout()
    
    
    ax5 = fig.add_subplot(1,5,5)
    ax5.plot([3,max_age],[3,max_age],'k:')

    sns.regplot(data=dt, x='AGE', y='hub_age_all', ax=ax5, color='darksalmon')
    
    if S == 'Macaca fascicularis' or S == 'Macaca mulatta':
        ax5.text(0.95, 0.03, err_5, horizontalalignment='right',verticalalignment='bottom',transform=ax5.transAxes)
    else:
        ax5.text(1, 0, err_5, horizontalalignment='right',verticalalignment='bottom',transform=ax5.transAxes)
    
    if S == 'Macaca fascicularis':
        ax5.text(12,25,'HUB_all', ha='center', va='center', size=20,color = 'darksalmon')
    if S == 'Normal Human':
        ax5.text(35,56,'HUB_all', ha='center', va='center', size=20,color = 'darksalmon') 
   
    plt.axis('equal')
    ax5.set_xlabel('True age (years)')
    #ax5.set_ylabel('Predicted age (years)')
    plt.ylabel('')
    if S == 'Macaca fascicularis' or S == 'Macaca mulatta':
        plt.ylim(0,25)       
    else:
        plt.ylim(20,Y_max)       
    plt.xlim(min_age,max_age)
    sns.despine()
    plt.tight_layout()

In [ ]:
age_graph(MF,8,20,'NA','Macaca fascicularis')
plt.show()

In [ ]:
age_graph(ML,3,23,'NA','Macaca mulatta')
plt.show()

In [ ]:
#age_graph(NHA_All,20,50,50,'Normal Human')
age_graph(NH,0,80,80,'Normal Human')
plt.show()

In [ ]:
#age_graph(NHA_All,20,50,50,'Normal Human')
age_graph(AH,0,80,80,'abnormal Human')
plt.show()